# Module 02 — Services & Parameters

## Before you start

Make sure the robot simulator is running:
```bash
python3 /home/jovyan/course_materials/scripts/robot_simulator.py
```

This module is a lab session. Observe first, understand after.

---

## 🎥 Watch First

<div style="padding:56.25% 0 0 0;position:relative;">
  <iframe src="https://player.vimeo.com/video/YOUR_VIDEO_ID"
          style="position:absolute;top:0;left:0;width:100%;height:100%;"
          frameborder="0" allow="autoplay; fullscreen; picture-in-picture" allowfullscreen>
  </iframe>
</div>

---

## Part 1 — Topics Have a Problem

### 🧪 Experiment 1.1 — Drive the robot, then stop it

Start the robot driving in a circle:
```bash
ros2 topic pub /cmd_vel geometry_msgs/msg/Twist "{linear: {x: 0.3}, angular: {z: 0.5}}"
```

Now imagine this is a real robot heading toward a wall.
You need to stop it **immediately** and get a confirmation that it stopped.

**Can you do that with a topic?**

> 📝 Write your answer before reading on.

Press Ctrl+C to stop publishing. Notice: you sent the stop but got **no response**. No confirmation. No acknowledgement. The robot just stopped — or didn't.

This is the fundamental limitation of topics: **fire and forget**. Perfect for streaming data. Useless when you need a response.

---

### 💡 This is exactly why Services exist

> A **Service** is a request-response interaction. You send a request, you wait, you get a confirmation back.

The robot simulator already has services built in. Let's explore them.

---

## Part 2 — Exploring Services

### 🧪 Experiment 2.1 — What services does the robot offer?

```bash
ros2 service list
```

You should see:
```
/emergency_stop
/reset_position
```

**Before calling either one:** What do you predict each will do to the robot?

> `/emergency_stop` — I predict: ___

> `/reset_position` — I predict: ___

---

### 🧪 Experiment 2.2 — Call the emergency stop

Start the robot moving first:
```bash
ros2 topic pub /cmd_vel geometry_msgs/msg/Twist "{linear: {x: 0.5}}" 
```

In a second terminal, call the service:
```bash
ros2 service call /emergency_stop std_srvs/srv/Empty {}
```

**Observe:**
- What happened to the robot in RViz?
- What did the terminal show you?
- Did you get a response? How is this different from a topic?

---

### 🧪 Experiment 2.3 — Try to drive after emergency stop

With the emergency stop active, try to send a velocity command:
```bash
ros2 topic pub /cmd_vel geometry_msgs/msg/Twist "{linear: {x: 0.5}}" --once
```

**What happens? Why?**

Now reset the robot:
```bash
ros2 service call /reset_position std_srvs/srv/Empty {}
```

Try driving again. **Does it work now?**

---

### 🧪 Experiment 2.4 — Inspect the service type

```bash
ros2 service type /emergency_stop
ros2 interface show std_srvs/srv/Empty
```

**What do you see?** The `Empty` service has no request data and no response data — just the call itself.

Now think: for a service that changes the robot speed limit and returns confirmation, what would the interface need to contain?

> 📝 Write your prediction.

---

### 💡 Conclusion: Services vs Topics

| | Topic | Service |
|--|-------|--------|
| Communication | One-way | Request → Response |
| Confirmation | None | Always |
| Use case | Continuous data | One-time operations |
| Example | `/odom`, `/cmd_vel` | `/emergency_stop`, take photo, calculate path |

---

## Part 3 — Write Your Own Service

You'll build a service that sets the robot's maximum speed — something meaningful for robot safety.

### Step 1 — Create the interface package


In [ ]:
%%bash
cd ~/ros2_ws/src
ros2 pkg create ros2_tutorials_interfaces --build-type ament_cmake
mkdir -p ros2_tutorials_interfaces/srv
# Remove unnecessary files
rm -f ros2_tutorials_interfaces/CMakeLists.txt ros2_tutorials_interfaces/package.xml
echo 'Interface package created!'


### Step 2 — Define the service interface

What goes in the request? What goes in the response?

```
Request  →  what the client sends
---        (separator)
Response ←  what the server sends back
```


In [ ]:
%%writefile ~/ros2_ws/src/ros2_tutorials_interfaces/srv/SetMaxSpeed.srv
# Request: new speed limits
float64 max_linear_speed
float64 max_angular_speed
---
# Response: confirmation
bool success
string message


### Step 3 — Configure the package

Create `CMakeLists.txt`:


In [ ]:
%%writefile ~/ros2_ws/src/ros2_tutorials_interfaces/CMakeLists.txt
cmake_minimum_required(VERSION 3.8)
project(ros2_tutorials_interfaces)

find_package(ament_cmake REQUIRED)
find_package(rosidl_default_generators REQUIRED)

rosidl_generate_interfaces(${PROJECT_NAME}
  'srv/SetMaxSpeed.srv'
)

ament_package()


In [ ]:
%%writefile ~/ros2_ws/src/ros2_tutorials_interfaces/package.xml
<?xml version="1.0"?>
<package format="3">
  <name>ros2_tutorials_interfaces</name>
  <version>0.0.1</version>
  <description>Custom interfaces</description>
  <maintainer email="student@rosforge.com">Student</maintainer>
  <license>Apache-2.0</license>
  <buildtool_depend>ament_cmake</buildtool_depend>
  <build_depend>rosidl_default_generators</build_depend>
  <exec_depend>rosidl_default_runtime</exec_depend>
  <member_of_group>rosidl_interface_packages</member_of_group>
</package>


### Step 4 — Write the service server


In [ ]:
%%writefile ~/ros2_ws/src/ros2_tutorials/ros2_tutorials/service_server.py
#!/usr/bin/env python3
import rclpy
from rclpy.node import Node
from ros2_tutorials_interfaces.srv import SetMaxSpeed

class SpeedLimitServer(Node):
    def __init__(self):
        super().__init__('speed_limit_server')
        self.max_linear = 1.0
        self.max_angular = 2.0
        self.srv = self.create_service(
            SetMaxSpeed, 'set_max_speed', self.handle_request)
        self.get_logger().info('Speed limit server ready!')

    def handle_request(self, request, response):
        if request.max_linear_speed <= 0 or request.max_angular_speed <= 0:
            response.success = False
            response.message = 'Speed must be positive'
            return response
        self.max_linear = request.max_linear_speed
        self.max_angular = request.max_angular_speed
        response.success = True
        response.message = f'Speed updated: linear={self.max_linear}, angular={self.max_angular}'
        self.get_logger().info(response.message)
        return response

def main(args=None):
    rclpy.init(args=args)
    rclpy.spin(SpeedLimitServer())
    rclpy.shutdown()

if __name__ == '__main__':
    main()


### Step 5 — Write the service client


In [ ]:
%%writefile ~/ros2_ws/src/ros2_tutorials/ros2_tutorials/service_client.py
#!/usr/bin/env python3
import sys, rclpy
from rclpy.node import Node
from ros2_tutorials_interfaces.srv import SetMaxSpeed

class SpeedLimitClient(Node):
    def __init__(self):
        super().__init__('speed_limit_client')
        self.client = self.create_client(SetMaxSpeed, 'set_max_speed')
        while not self.client.wait_for_service(timeout_sec=1.0):
            self.get_logger().info('Waiting for server...')

    def send_request(self, linear, angular):
        req = SetMaxSpeed.Request()
        req.max_linear_speed = linear
        req.max_angular_speed = angular
        future = self.client.call_async(req)
        rclpy.spin_until_future_complete(self, future)
        return future.result()

def main(args=None):
    rclpy.init(args=args)
    client = SpeedLimitClient()
    linear = float(sys.argv[1]) if len(sys.argv) > 1 else 1.0
    angular = float(sys.argv[2]) if len(sys.argv) > 2 else 2.0
    result = client.send_request(linear, angular)
    print(f'Response: {result.message}')
    client.destroy_node()
    rclpy.shutdown()

if __name__ == '__main__':
    main()


Add to `setup.py` entry_points:
```python
'py_speed_server = ros2_tutorials.service_server:main',
'py_speed_client = ros2_tutorials.service_client:main',
```

Add to `package.xml` in `ros2_tutorials`:
```xml
<depend>ros2_tutorials_interfaces</depend>
```


In [ ]:
%%bash
cd ~/ros2_ws && colcon build && source install/setup.bash
echo 'Build complete!'


### 🧪 Experiment 3.1 — Test your service

**Terminal 1:** Start the server
```bash
ros2 run ros2_tutorials py_speed_server
```

**Terminal 2:** Call it with different values
```bash
# Set a very low speed
ros2 run ros2_tutorials py_speed_client 0.1 0.5

# Try a negative value — what happens?
ros2 run ros2_tutorials py_speed_client -1.0 1.0

# Or call directly via CLI
ros2 service call /set_max_speed ros2_tutorials_interfaces/srv/SetMaxSpeed "{max_linear_speed: 2.0, max_angular_speed: 3.0}"
```

**The client blocks and waits for the response.** This is the key difference from a topic publish.

---

## Part 4 — Parameters

### 🧪 Experiment 4.1 — Discover the robot's parameters

```bash
ros2 param list /mobile_robot
```

You should see:
```
max_angular_speed
max_linear_speed
publish_rate
```

**Prediction:** If you change `max_linear_speed` to 0.1 and then send `/cmd_vel` with `x: 2.0`, what speed will the robot actually move at?

> 📝 Write your prediction.

---

### 🧪 Experiment 4.2 — Change a parameter at runtime

First get the current value:
```bash
ros2 param get /mobile_robot max_linear_speed
```

Now change it:
```bash
ros2 param set /mobile_robot max_linear_speed 0.1
```

Now send a high-speed command:
```bash
ros2 topic pub /cmd_vel geometry_msgs/msg/Twist "{linear: {x: 5.0}}" --once
```

Watch `/odom`. **Was your prediction correct?**

---

### 🧪 Experiment 4.3 — What happens after a restart?

Stop the simulator (Ctrl+C) and restart it:
```bash
python3 /home/jovyan/course_materials/scripts/robot_simulator.py
```

Check the parameter again:
```bash
ros2 param get /mobile_robot max_linear_speed
```

**Is it still 0.1 or back to 1.0? What does this tell you about parameters?**

> 📝 Write your conclusion.

---

### Write a node with parameters


In [ ]:
%%writefile ~/ros2_ws/src/ros2_tutorials/ros2_tutorials/velocity_publisher.py
#!/usr/bin/env python3
import rclpy
from rclpy.node import Node
from geometry_msgs.msg import Twist

class VelocityPublisher(Node):
    def __init__(self):
        super().__init__('velocity_publisher')
        # Declare parameters with defaults
        self.declare_parameter('linear_speed', 0.3)
        self.declare_parameter('angular_speed', 0.5)
        self.declare_parameter('timer_period', 0.5)

        self.publisher_ = self.create_publisher(Twist, 'cmd_vel', 10)
        period = self.get_parameter('timer_period').value
        self.timer = self.create_timer(period, self.timer_callback)
        self.get_logger().info('Velocity publisher with parameters started!')

    def timer_callback(self):
        msg = Twist()
        msg.linear.x = self.get_parameter('linear_speed').value
        msg.angular.z = self.get_parameter('angular_speed').value
        self.publisher_.publish(msg)

def main(args=None):
    rclpy.init(args=args)
    rclpy.spin(VelocityPublisher())
    rclpy.shutdown()

if __name__ == '__main__':
    main()


### 🧪 Experiment 4.4 — Override parameters at launch

```bash
cd ~/ros2_ws && source install/setup.bash

# Default circle
ros2 run ros2_tutorials py_velocity_publisher

# Tight circle
ros2 run ros2_tutorials py_velocity_publisher --ros-args -p linear_speed:=0.1 -p angular_speed:=2.0

# Straight line
ros2 run ros2_tutorials py_velocity_publisher --ros-args -p linear_speed:=0.5 -p angular_speed:=0.0
```

**No code changes needed.** Same node, completely different behavior.

Now change the speed while the node is running:
```bash
ros2 param set /velocity_publisher linear_speed 0.8
```

**What happened? Did it change immediately?**

---

## 🧠 Conceptual Challenges

**1.** Your robot needs to stream battery level every second to a dashboard, and also accept a command to return to the charging station. Which mechanism do you use for each? Why?

**2.** A service call to plan a navigation path takes 3 seconds. During those 3 seconds, can your node do anything else? What does this mean for robot responsiveness?

**3.** You set `max_linear_speed` to 0.5 via `ros2 param set`. A teammate restarts the simulator. Is the parameter preserved? How would you make it persistent?

**4.** Your service server receives a request but the calculation takes 10 seconds. The client is waiting. Should the client have a timeout? What happens if it doesn't?

**5.** Name one situation where you would **not** use a service even though you need a response.

---

## 📚 Summary

✅ **Services** are for request-response — one-time operations that need confirmation

✅ **Topics** stream continuously — no confirmation, no waiting

✅ **Parameters** configure nodes without changing code — but reset on restart by default

✅ Parameters can be overridden at launch with `--ros-args -p name:=value`

➡️ **Next:** Module 03 — Launch Files & Workspace Organization